In [1]:
import pandas as pd

#  variables to set up the load and write of data files
path = 'C:/Users/alexj/TradingView'
file = 'XRPUSD_1D_07082024_normalized.csv'
asset = 'XRPUSD'
period = '1D'
endpoint = '07082024'

#  the complete path of the file to be loaded
load_data = path+'/'+file

#  pandas reading the csv file
df = pd.read_csv(load_data)

#  limiting the new dataset to only columns of interest
dfwk = df[['date', 'close', 'Volume', 'RSI', 'weekday']]

#  filtering the new dataset to only show Sundays
dfsun = dfwk.query('weekday == "Sunday"').copy()

In [2]:
"""
loaded feature definitions:

dfwk['date'] = the date of the events
dfwk['close'] = the closing proce of the asset on the date referenced
dfwk['volume'] = the transaction volume on the date referenced
dfwk['RSI'] = the relative strenght index associated with the date referenced
dfwk['weekday'] = the weekday name of the date referenced

"""

#  added features for filtering and analysis

#  ensure that date is datetime value
dfsun['date'] = pd.to_datetime(df['date'])

#  the year of the current record.
dfsun['year'] = dfsun['date'].dt.strftime('%Y')

#  the month name of the current record.
dfsun['month_name'] = dfsun['date'].dt.strftime('%b')

#  the month number of the current record.
dfsun['month_num'] = dfsun['date'].dt.month

#  the week of the year that the current record is in.
dfsun['woy'] = dfsun['date'].dt.strftime('%U')

#  the close on the previous Sunday.
dfsun['pwk_close'] = dfsun.close.shift(1)

#  the difference between current and previous Sunday close.
dfsun['pwk_diff'] = dfsun.close - dfsun.pwk_close

#  the direction of the difference between current and previous Sunday close.
dfsun['direction'] = dfsun['pwk_diff'].apply(lambda x: 'pos' if x >= 0 else 'neg')

#  the direction of the difference between previous Sunday close and its previous.
dfsun['prev_dir'] = dfsun['direction'].shift(1)

#  True if the direction is the same as previous.
dfsun['continue'] = dfsun['direction'] == dfsun['prev_dir']


#  show the first 15 rows
dfsun.head(15)

,date,close,Volume,RSI,weekday,year,month_name,month_num,woy,pwk_close,pwk_diff,direction,prev_dir,continue
0,2014-12-14,0.0173,9187.000000,75.338849,Sunday,2014,Dec,12,50,NaN,NaN,neg,None,False
7,2014-12-21,0.0248,15842.000000,71.469785,Sunday,2014,Dec,12,51,0.0173,0.0075,pos,neg,False
14,2014-12-28,0.0237,9743.000000,61.792697,Sunday,2014,Dec,12,52,0.0248,-0.0011,neg,pos,False
21,2015-01-04,0.0190,81253.000000,42.309093,Sunday,2015,Jan,1,01,0.0237,-0.0047,neg,neg,True
28,2015-01-11,0.0188,22814.000000,41.603796,Sunday,2015,Jan,1,02,0.0190,-0.0002,neg,neg,True
35,2015-01-18,0.0157,65575.000000,36.547114,Sunday,2015,Jan,1,03,0.0188,-0.0031,neg,neg,True
42,2015-01-25,0.0167,54021.000000,42.330532,Sunday,2015,Jan,1,04,0.0157,0.0010,pos,neg,False
49,2015-02-01,0.0142,55668.000000,31.491313,Sunday,2015,Feb,2,05,0.0167,-0.0025,neg,pos,False
56,2015-02-08,0.0145,26112.000000,41.167905,Sunday,2015,Feb,2,06,0.0142,0.0003,pos,neg,False
63,2015-02-15,0.0139,103149.000000,40.989585,Sunday,2015,Feb,2,07,0.0145,-0.0006,neg,pos,False


In [3]:
#  export the new dataset to a .csv file in the smae directory as the notebook
dfsun.to_csv(f'{asset}_{period}_{endpoint}_weekly.csv', index=False)